In [41]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import sklearn.metrics as sm 
import time
from tqdm import tqdm
from timeit import default_timer as timer
import time
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV

def iqr(data)  
    # First quartile (Q1)
    Q1 = np.percentile(data, 25, interpolation = 'midpoint')

    # Third quartile (Q3)
    Q3 = np.percentile(data, 75, interpolation = 'midpoint')

    # Interquaritle range (IQR)
    IQR = Q3 - Q1

    return Q1,Q3,IQR

# Read the parquet file for the dataset

##### Compressing the data set by changing the customer id to int64 and to hex 

In [13]:
def read_parquet(path='', cols=None):
    """
        Returns pandas dataframe

        Parameters
        -----------
        path to training data in parquet format

        Returns
        ----------
        Pandas DataFrame

        """
    # LOAD DATAFRAME
    if cols is not None:
        dataFrame = pd.read_parquet(path, columns=cols)
    else:
        dataFrame = pd.read_parquet(path)

    
    dataFrame['customer_ID'] = dataFrame['customer_ID'].str[-16:].apply(int, base=16).astype('int64')
    dataFrame.S_2 = pd.to_datetime(dataFrame.S_2)

    
    dataFrame = dataFrame.fillna(-127)
    print('shape of data:', dataFrame.shape)

    return dataFrame


print('Reading train data...')
TRAIN_PATH = 'train.parquet'
train = read_parquet(path=TRAIN_PATH)

Reading train data...
shape of data: (5531451, 190)


# Feature engineering is done seperately for numerical columns and categorical columns and then groupby using the date as there are a lot a record for each customer for seperate statements. Agreegation is done using min,max,last,std, count etc

In [15]:
def feature_engineering(df):
    """
    :param df: pandas dataframe of train dataset
    :return: feature engineered data
    
    """
    all_col_of_df = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    num_features = [col for col in all_col_of_df if col not in cat_features]

    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

    df = pd.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg
    print('shape after engineering', df.shape)

    return df


train = feature_engineering(train)


shape after engineering (458913, 918)


# The target variable is added to the groupby dataset by merging common records.

In [16]:
# ADD TARGETS
output_var = pd.read_csv('C:/Users/shant/Downloads/train_labels.csv')
output_var['customer_ID'] = output_var['customer_ID'].str[-16:].apply(int, base=16).astype('int64')
output_var = output_var.set_index('customer_ID')
train = train.merge(output_var, left_index=True, right_index=True, how='left')
train.target = train.target.astype('int8')
del output_var


train = train.sort_index().reset_index()

# FEATURES
FEATURES = train.columns[1:-1]
print(f'There are {len(FEATURES)} features!')

There are 918 features!


In [18]:
train.isnull().sum()[train.isnull().sum() != 0].index

Index(['P_2_std', 'D_39_std', 'B_1_std', 'B_2_std', 'R_1_std', 'S_3_std',
       'D_41_std', 'B_3_std', 'D_42_std', 'D_43_std',
       ...
       'D_136_std', 'D_137_std', 'D_138_std', 'D_139_std', 'D_140_std',
       'D_141_std', 'D_142_std', 'D_143_std', 'D_144_std', 'D_145_std'],
      dtype='object', length=177)

# Since missing value count is less then 1% so missing value data are removed

In [19]:
shape_before_remove = train.shape[0]
train = train.dropna()
print('Number of records removed: ',abs(shape_before_remove - train.shape[0]))
print('% of customers removed from data: ', abs(shape_before_remove - train.shape[0]) * 100 /shape_before_remove)

Number of records removed:  5120
% of customers removed from data:  1.1156798783211632


In [20]:
X = train.drop("target",axis=1)
y= train["target"]
X.describe()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
count,4.537930e+05,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,...,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000
mean,-3.857061e+15,-0.836002,1.831865,-4.517465,0.143169,-0.089036,4.986713,5.725742,0.158920,16.885796,...,1.072941,12.178088,1.169522,1.436225,12.178088,-0.769785,1.038445,12.178088,4.928040,1.602277
std,5.328744e+18,10.446057,9.666892,25.002823,8.539724,9.582557,5.424514,5.302279,1.342613,16.069912,...,0.287971,2.344432,1.376439,0.592898,2.344432,0.638304,0.194184,2.344432,1.492104,0.734291
min,-9.223358e+18,-127.000000,0.000000,-127.000000,-127.000000,-127.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,-1.000000,1.000000,2.000000,-1.000000,1.000000,2.000000,-1.000000,1.000000
25%,-4.620310e+18,0.448502,0.022928,0.356623,0.549564,0.445101,0.538462,0.960769,0.000000,3.000000,...,1.000000,13.000000,0.000000,1.000000,13.000000,-1.000000,1.000000,13.000000,4.000000,1.000000
50%,2.976669e+15,0.671380,0.038433,0.595314,0.747344,0.681235,3.615385,5.547002,0.000000,16.000000,...,1.000000,13.000000,0.000000,1.000000,13.000000,-1.000000,1.000000,13.000000,6.000000,1.000000
75%,4.612774e+18,0.854776,0.067619,0.796277,0.907235,0.861990,7.692308,8.166536,0.000000,23.000000,...,1.000000,13.000000,3.000000,2.000000,13.000000,-1.000000,1.000000,13.000000,6.000000,2.000000
max,9.223350e+18,1.009089,90.515988,1.008194,1.010000,1.009998,156.000000,103.944697,151.000000,183.000000,...,6.000000,13.000000,3.000000,5.000000,13.000000,1.000000,3.000000,13.000000,6.000000,6.000000


def outlierdetection(df):
    
    q1=df.quantile(0.25)

    q3=df.quantile(0.75)

    IQR=q3-q1

    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

    return outliers 
        
        
    
        

train.to_csv('C:/Users/aagarw16/OneDrive - stevens.edu/Desktop/ML Project/Project/Data_transformed.csv')

# We apply Robust Scaling to the data set to get the outliers into the range 

In [22]:
X_transform= RobustScaler().fit_transform(X)


In [23]:
X_transform


array([[-0.999269  , -0.62891364,  0.41871238, ...,  0.        ,
        -1.5       ,  1.        ],
       [-0.99925109,  0.74503548, -0.56698068, ...,  0.        ,
         0.        ,  0.        ],
       [-0.99925073,  0.32260794, -0.00912282, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.99861823, -0.75459455,  0.8142958 , ...,  0.        ,
        -1.5       ,  1.        ],
       [ 0.99862279,  0.43691329,  0.47225463, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.99862332, -1.07944177,  0.58651556, ...,  0.        ,
         0.        ,  0.        ]])

In [24]:
X_transform1 = pd.DataFrame(X_transform,columns=X.columns)

In [26]:
X_transform1.describe()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
count,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,...,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000,453793.000000
mean,-0.000740,-3.710255,40.129720,-11.629391,-1.689170,-1.847658,0.191691,0.024805,0.158920,0.044290,...,0.072941,-0.821912,0.389841,0.436225,-0.821912,0.230215,0.038445,-0.821912,-0.535980,0.602277
std,0.577136,25.702302,216.305759,56.806572,23.907053,23.015027,0.758265,0.735838,1.342613,0.803496,...,0.287971,2.344432,0.458813,0.592898,2.344432,0.638304,0.194184,2.344432,0.746052,0.734291
min,-0.999269,-314.249453,-0.859969,-290.217592,-357.164425,-306.270937,-0.505376,-0.769800,0.000000,-0.800000,...,0.000000,-11.000000,-0.333333,0.000000,-11.000000,0.000000,0.000000,-11.000000,-3.500000,0.000000
25%,-0.500731,-0.548590,-0.346939,-0.542908,-0.552966,-0.566419,-0.430108,-0.636467,0.000000,-0.650000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.499269,0.451410,0.653061,0.457092,0.447034,0.433581,0.569892,0.363533,0.000000,0.350000,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,0.998623,0.831236,2024.520064,0.939102,0.734351,0.788610,21.301075,13.655409,151.000000,8.350000,...,5.000000,0.000000,1.000000,4.000000,0.000000,2.000000,2.000000,0.000000,0.000000,5.000000


# We use IQR method to find the number of columns having outliers

In [27]:
Q1_1 = X_transform1.quantile(0.25)
Q3_1 = X_transform1.quantile(0.75)
IQR_1 = Q3_1 - Q1_1
y_value1=((X_transform1 < (Q1_1 - 1.5 * IQR_1)) | (X_transform1 > (Q3_1 + 1.5 * IQR_1))).sum()

# All columns where having outliers

In [28]:
((y_value1/X_transform1.shape[0])>1).value_counts()

False    919
dtype: int64

# Finding the Percentage of outliers in each columns 

In [29]:
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
y_value=((((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).sum())/X.shape[0])


In [30]:
y_value

customer_ID     0.000000
P_2_mean        0.040089
P_2_std         0.083064
P_2_min         0.040250
P_2_max         0.005522
                  ...   
D_66_last       0.115108
D_66_nunique    0.038075
D_68_count      0.149317
D_68_last       0.014641
D_68_nunique    0.015798
Length: 919, dtype: float64

# Calculating the number of columns having outlier greater then 15%

In [31]:
(((y_value1/X_transform1.shape[0])*100)>15).value_counts()

False    713
True     206
dtype: int64

# 216 columns have outliers more then 15% and rest less then 15% so we could apply ANN and other linear algorithm 

In [34]:
(((y_value/X.shape[0])*100)>15)

customer_ID     False
P_2_mean        False
P_2_std         False
P_2_min         False
P_2_max         False
                ...  
D_66_last       False
D_66_nunique    False
D_68_count      False
D_68_last       False
D_68_nunique    False
Length: 919, dtype: bool

In [ ]:
X_transform1


In [ ]:
X

# Train Test Spliting done on the X_Transformed after applying Robust Scaling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_transform1,y,test_size=0.3, random_state=30)

# Training using SVM classifier 

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

# With the available resources due to the high computational requirement of SVM for multidimensional data SVM even after running for 72 hours it was still excuting which is not a feasible model as for hyper parameter tuning the model would require to much time and computation. Also outliers where not handled due to no concrete method for handling them currently for linear model. So SVM was replaced with another Tree based algorithm XGboost which is Outlier invarient and also works very well on high dimension data with added advantage of learning from prior and adjusting the training in each tree.